# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [11]:
import numpy as np
import cv2 as cv2
import glob

def getK():
    return np.array([[7.188560e+02, 0.000000e+00, 6.071928e+02],
                     [0, 7.188560e+02, 1.852157e+02],
                     [0, 0, 1]])

def getTruePose():
    file = '00.txt'
    return np.genfromtxt(file, delimiter=' ', dtype=None)

<details>
  <summary>tips:</summary>

- [Feature Matching](https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html)
- To get the keypoint of a certain match do: ```kp1[match.queryIdx].pt``` and ```kp2[match.trainIdx].pt```

In [12]:
# Function to extract and match keypoints using SIFT
def extract_keypoints_sift(img1, img2, K, baseline):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect and compute descriptors for both images
    kp1, des1 = sift.detectAndCompute(img1, None)  # Keypoints and descriptors for first image
    kp2, des2 = sift.detectAndCompute(img2, None)  # Keypoints and descriptors for second image

    # Create BFMatcher object with cross-check enabled for better matches
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)  # Match descriptors between images
    matches = sorted(matches, key=lambda x: x.distance)  # Sort matches by distance (quality)

    # Extract matched points from keypoints
    match_points1 = np.array([kp1[m.queryIdx].pt for m in matches], dtype='float32')
    match_points2 = np.array([kp2[m.trainIdx].pt for m in matches], dtype='float32')

    # Triangulate points to obtain 3D coordinates
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))  # Projection matrix for left image
    M_right = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))  # Projection matrix for right image

    # Prepare points for triangulation
    p1_flip = np.vstack((match_points1.T, np.ones((1, match_points1.shape[0]))))
    p2_flip = np.vstack((match_points2.T, np.ones((1, match_points2.shape[0]))))

    # Perform triangulation
    P = cv2.triangulatePoints(M_left, M_right, p1_flip[:2], p2_flip[:2])
    P /= P[3]  # Normalize homogeneous coordinates

    land_points = P[:3].T  # Convert to (N, 3) array of 3D points

    return land_points, match_points1  # Return 3D points and 2D points from the first image

In [13]:
# Function to track features from one image to the next
def featureTracking(prev_img, next_img, prev_points, world_points):
    # Parameters for optical flow
    params = dict(winSize=(21, 21),
                  maxLevel=3,
                  criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

    # Track points using optical flow
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, **params)
    status = status.flatten()  # Flatten status array for easier indexing

    # Filter points that were successfully tracked
    good_prev_points = prev_points[status == 1]  # Points in the previous image that were tracked
    good_next_points = next_points[status == 1]  # Corresponding points in the next image
    good_world_points = world_points[status == 1]  # Corresponding 3D world points

    return good_world_points, good_prev_points, good_next_points  # Return filtered points

In [14]:
# Function to play the image sequence and estimate camera pose
def playImageSequence(l_img_paths, r_img_paths, K, baseline):
    baseline = 0.54
    reference_img = cv2.imread(l_img_paths[0], 0)  # Load the initial reference image
    truePose = getTruePose()  # Load ground truth poses
    traj = np.zeros((600, 600, 3), dtype=np.uint8)  # Initialize an image for trajectory visualization
    maxError = 0  # Track maximum error

    for i in range(1, len(l_img_paths)):
        curImage = cv2.imread(l_img_paths[i], 0)  # Load current left image
        curImage_R = cv2.imread(r_img_paths[i], 0)  # Load current right image

        # Step 1.2 and 1.3: Extract 3D and 2D correspondences
        if i == 1:
            # Initial feature extraction for the first pair of images
            landmark_3D, reference_2D = extract_keypoints_sift(reference_img, curImage_R, K, baseline)
        else:
            # Track features between the previous and current images
            landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, curImage, reference_2D, landmark_3D)

        # Step 2.3: Estimate pose using PnP (Perspective-n-Point)
        success, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, None)

        if success:
            # Convert rotation vector to rotation matrix
            rot, _ = cv2.Rodrigues(rvec)
            tvec = -rot.T.dot(tvec)  # Convert camera pose to world coordinates
            inv_transform = np.hstack((rot.T, tvec))  # Create transformation matrix
            reference_img = curImage  # Update reference image for the next iteration

            # Step 2.4: Triangulate new feature points
            landmark_3D_new, reference_2D_new = extract_keypoints_sift(reference_img, curImage_R, K, baseline)
            reference_2D = reference_2D_new.astype('float32')  # Update 2D reference points
            # Transform new 3D points to world coordinates
            landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0]))))).T

            # Visualize the estimated pose and trajectory
            draw_x, draw_y = int(tvec[0]) + 300, 600 - (int(tvec[2]) + 100)  # Convert coordinates for visualization
            true_x, true_y = int(truePose[i][3]) + 300, 600 - (int(truePose[i][11]) + 100)  # Ground truth coordinates

            # Calculate current error
            curError = np.sqrt((tvec[0] - truePose[i][3]) ** 2 + (tvec[1] - truePose[i][7]) ** 2 + (tvec[2] - truePose[i][11]) ** 2)
            maxError = max(maxError, curError)  # Update maximum error

            # Draw trajectory on the visualization image
            cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2)  # Draw estimated position
            cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2)  # Draw ground truth position
            text = f"Coordinates: x ={tvec[0]:.2f}m y = {tvec[1]:.2f}m z = {tvec[2]:.2f}m"  # Display coordinates
            cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8)  # Overlay text
            vis = np.hstack((traj, np.dstack((curImage, curImage, curImage))))  # Combine trajectory and current image
            cv2.imshow("Trajectory", vis)  # Show visualization
            if cv2.waitKey(1) & 0xFF == 27:  # Break loop if 'Esc' is pressed
                break

In [15]:
# Load image paths
left_img_paths = sorted(glob.glob('left/*.png'))
right_img_paths = sorted(glob.glob('right/*.png'))

K = getK()
playImageSequence(left_img_paths, right_img_paths, K, baseline)

UnboundLocalError: cannot access local variable 'tracked_2Dpoints' where it is not associated with a value

# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: The following function `removeDuplicate` can be used for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.

In [ ]:
def removeDuplicate(queryPoints, refPoints, radius=5):
    # remove duplicate points from new query points,
    for i in range(len(queryPoints)):
        query = queryPoints[i]
        xliml, xlimh = query[0] - radius, query[0] + radius
        yliml, ylimh = query[1] - radius, query[1] + radius
        inside_x_lim_mask = (refPoints[:, 0] > xliml) & (refPoints[:, 0] < xlimh)
        curr_kps_in_x_lim = refPoints[inside_x_lim_mask]

        if curr_kps_in_x_lim.shape[0] != 0:
            inside_y_lim_mask = (curr_kps_in_x_lim[:, 1] > yliml) & (curr_kps_in_x_lim[:, 1] < ylimh)
            curr_kps_in_x_lim_and_y_lim = curr_kps_in_x_lim[inside_y_lim_mask, :]
            if curr_kps_in_x_lim_and_y_lim.shape[0] != 0:
                queryPoints[i] = np.array([0, 0])
    return (queryPoints[:, 0] != 0)